# Get sequences
of the species that are in the numt dataset

In [3]:
import os 
import pandas as pd 
import numpy as np
from bold import boldAPI

In [8]:


dna_data_path = os.path.join('data', 'dna_sequences.csv')
if not os.path.exists(dna_data_path):
    dna_df = pd.read_csv(dna_data_path)
    unique_species = dna_df['Species'].unique()
    vfunc = np.vectorize(boldAPI.get_sequences)
    results = pd.concat(vfunc(unique_species))
else: 
    results = pd.read_csv(dna_data_path)

results = results[results['Sequence'].str.contains('N') == False] # if there is an N its an error
results['Sequence'] = results['Sequence'].apply(lambda x: x.replace('-', '')) # we dont care about the - its an alignment artifact and we can just remove them
results = results.drop_duplicates(subset='Sequence') # there are a lot of duplicate sequences just drop all but the first

results.to_csv(dna_data_path, index=False)



In [6]:
from bold import boldAPI
import importlib
importlib.reload(boldAPI)

boldAPI.get_sequences('Acropora millepora')

Acropora millepora


,Species,BOLDProcessId,Sequence
0,Acropora millepora,GBCI10386-19,TACGTTATATTTAGTCTTTGGGATTGGGGCAGGCATGATTGGCACG...
0,Acropora millepora,GBFSE20474-24,TGCTTTTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE25693-24,GCTTTTACCCATTGAGGTGGGTGAGTTGGAGTGGTTGCGGCCTGCA...
0,Acropora millepora,GBFSE26948-24,TGCTTTTACCCAGTTACTTGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE28464-24,TGCTTCTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE31107-24,TGCTTCTACCCAATTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE35612-24,TGCTTCTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE43179-24,TGCTTTTACCCAGTTACATGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE45768-24,TGCTTCTACCCAGTTACACGGGTGAGTTGGAGTGGTCGCGGCCTGC...
0,Acropora millepora,GBFSE47451-24,GTTTTTACCCGGTTATTCGGGTGAGTTGGAGTGGTCGCGGCCTGCC...


In [10]:
results

,Species,BOLDProcessId,Sequence
0,Abscondita terminalis,GBCL39854-19,CTTAATAAATGACTTTTTTCAACAAATCATAAAGACATTGGAACTT...
1,Abscondita terminalis,SPMIS002-22,AACTTTATATTTTATCTTCGGAATCTGAGCAGGAATATTAGGAACC...
2,Actias luna luna,GBGLS176-19,TTTTGATTTTTTGGGCATCCAGAAGTTTATATTTTAATTTTACCAG...
3,Actias luna,LGSM242-04,AACTTTATATTTTATTTTTGGAATTTGAGCAGGAATAGTAGGAACT...
4,Actias luna,LGSMC795-05,AACTTTATATTTTATTTTTGGAATTTGAGCAGGAATAGTAGGAACT...
...,...,...,...
41944,Zootermopsis nevadensis,GBMTG5022-16,ATGTTAATATTTTTTTATTTTTCTACCTTTTTTCTTTGTGGTTTGT...
41945,Zootermopsis nevadensis,GBMTG5022-16,ATGAATAAACCCATACGAAAAAACCACCCACTAATCAAAATTATAA...
41946,Zootermopsis nevadensis,GBMH10124-19,GAACAGAACTTGGACAACCAGGATCCTAATTGGAGACGACCAAATT...
41947,Zootermopsis nevadensis nuttingi,GBMH9919-19,GAACAGAACTTGGACAACCAGGATCCTAATTGGAGACGACCAAATT...


# get amino acid sequences for the DNA seqs

In [4]:
import os
import pandas as pd
from plmcluster import proteinextraction
# sequencePath = os.path.join('data', 'BOLD_sequences.csv')
# if os.path.exists(sequencePath) and:
#     results = pd.read_csv(sequencePath)

In [12]:
results.drop_duplicates(subset='Sequence', inplace=True) # there is a chance that after N and - being removed there are sequences that are duplicates now
results['AminoAcidSeq'] = results['Sequence'].apply(proteinextraction.get_aminoacid_seq)
results.to_csv(os.path.join('data', 'BOLD_seqs_with_protein.csv'), index=False)

/home/jnicolow/.conda/envs/numt/lib/python3.11/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(
